In [2]:
#Capacitated Warehouse location problem
link1 = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap82.txt'
link2 = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap113.txt'
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    response = urllib.request.urlretrieve(url,filename)
    file = open(filename,'r')
    return file

In [4]:
file = load_data(link2, 'cap113.txt')
data = file.readlines()

In [5]:
[locations, customers] = [int(x) for x in data[0].split(' ')[1:-1]]
capacity = []
fixcost = []
for i in range(locations):
    [c, f] = [float(x) for x in data[i+1].split(' ')[1:-1]]
    capacity.append(c)
    fixcost.append(f)
demand = []
varcost = np.zeros((locations,customers))
for j in range(customers):
    demand.append(float(data[j*9 + (1+locations)].split(' ')[1]))
    v = list(np.concatenate([data[j*9 + (1+locations+t)].split(' ')[1:-1] for t in range(1,9)]))
    for i in range(locations):
        varcost[i,j] = float(v[i])/demand[j] #cost of allocating demand of j to warehouse i per unit
        
        
print(locations)
print(customers)
print(capacity)
print(fixcost)
print(demand)
print(varcost)

50
50
[5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0]
[17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 0.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0]
[146.0, 87.0, 672.0, 1337.0, 31.0, 559.0, 2370.0, 1089.0, 33.0, 32.0, 5495.0, 904.0, 1466.0, 143.0, 615.0, 564.0, 226.0, 3016.0, 253.0, 195.0, 38.

In [ ]:
# based on https://www.researchgate.net/publication/23547095

In [17]:
#solution encode = facility opening decision
#I0 and I1, where I0 consists of the indices of the facilities that are closed and
#I1 consists of the indices of the facilities that are open
close_loc = set() #I0
open_loc = set() #I1
allocation = np.zeros((locations, customers))
solution = [close_loc, open_loc] #solution[0] = close, solution[1] = open

In [ ]:
# a move = taking one element from I0 and placing it into I1 (add move) or vice versa (drop move)
def add_move(solution, i):
    solution[0].remove(i) 
    solution[1].add(i)
    return solution

def drop_move(solution, i):
    solution[0].add(i)
    solution[1].remove(i)
    return solution

In [32]:
#define feasible condition: capacity >= demand
def feasible(solution, demand, capacity):
    check = sum([capacity[i] for i in solution[0]]) >= sum(demand)
    return check

In [36]:
# a set of neighborhood N(S) = {solutions obtained by applying a single local transformation (move, i.e. add or drop) to S}
# new solution must be feasible
def neighborhood(solution, demand, capacity):
    nbh = []
    for i in solution[0]: #open a closed location
        temp = add_move(solution, i)
        if feasible(temp, demand, capacity):
            nbh.append(temp)
    for i in solution[1]: #close an opened location
        temp = drop_move(solution, i)
        if feasible(temp, demand, capacity):
            nbh.append(temp)
    
    return nbh

False

In [8]:
def transportation ():
    model0 = gp.Model()
    x0 = {}
    for i in range(locations):
        for j in range(customers):
            x0[i,j] = model0.addVar(obj=varcost[i][j])

    #demand constraint
    demand_constr = {} 
    for j in range(customers):
        demand_constr[j] = model0.addConstr(sum(x0[i,j] for i in range(locations)) == demand[j])

    #capacity constraint
    cap_constr = {}
    for i in range(locations):
        cap_constr[i] = model0.addConstr(sum(x0[i,j] for j in range(customers)) <= capacity[i])

    #logical constraint
    log_constr = {}
    for i in range(locations):
        for j in range(customers):
            log_constr[i,j] = model0.addConstr(x0[i,j] <= y0[i,j] * min(capacity[i], demand[j]))

    model0.optimize()

    print("Objective: " + str(model0.objVal))

    print("Solution:")
    import pandas as pd
    index = ['Source ' + str(x) for x in range(locations)]
    columns = ['Customer ' + str(x) for x in range(customers)]

    solution = pd.DataFrame(index=index, columns=columns)

    for i in range(locations):
        for j in range(customers):
            solution.iloc[i,j] = x0[i,j].x

    print(solution)

In [37]:
#starting solution = open all facilities
open_loc.update([x for x in range(locations)])
print(solution)

({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49}, set())
